In [ ]:
# set Keras backend 
import os 
os.environ["KERAS_BACKEND"] = "torch"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np

# set warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# import modules and components
from FEXT.commons.utils.dataloader.generators import ML_model_dataloader
from FEXT.commons.utils.dataloader.serializer import DataSerializer, ModelSerializer
from FEXT.commons.utils.learning.inference import ImageEncoding
from FEXT.commons.utils.validation.images import ImageReconstruction
from FEXT.commons.utils.validation.checkpoints import ModelEvaluationSummary
from FEXT.commons.utils.validation.reports import evaluation_report
from FEXT.commons.constants import CONFIG

## Generate evaluation summary for all checkpoints

In [ ]:
summarizer = ModelEvaluationSummary()
checkpoints_summary = summarizer.checkpoints_summary()

## Load data and model

---


In [ ]:
# selected and load the pretrained model, then print the summary 
modelserializer = ModelSerializer()         
model, configuration, history, checkpoint_path = modelserializer.select_and_load_checkpoint()
model.summary(expand_nested=True)

# isolate the encoder from the autoencoder model, and use it for inference     
encoder = ImageEncoding(model, configuration)
encoder_model = encoder.encoder_model

In [4]:
# extract paths
dataserializer = DataSerializer(configuration)
train_data, validation_data = dataserializer.load_preprocessed_data(checkpoint_path)

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators    
eval_batch_size = CONFIG["evaluation"]["BATCH_SIZE"]
train_dataset, validation_dataset = ML_model_dataloader(train_data, validation_data, configuration,
                                                           batch_size=eval_batch_size)

# Model performance evaluation

---


### Evaluation of loss and metrics

We use huber loss for the Convolutional Auto Encoder (CAE) and cosine similarity as a metric to evaluate similarity between input and reconstructed output.

In [ ]:
evaluation_report(model, train_dataset, validation_dataset) 

### Reconstruction evaluation

Compare reconstructed images to original pictures to qualitatively evaluate the performance of the FeXT autoencoder model

In [ ]:
# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators    
train_dataset, validation_dataset = ML_model_dataloader(train_data, validation_data, configuration, batch_size=6)

# perform visual validation for the train dataset (initialize a validation tf.dataset
# with batch size of 10 images)
validator = ImageReconstruction(model)
validator.visualize_reconstructed_images(train_dataset, 'visual_evaluation_train', checkpoint_path)
validator.visualize_reconstructed_images(validation_dataset, 'visual_evaluation_val', checkpoint_path)

Visualize the original image and the reconstructed images, together with the corresponding features vector. The raw vector has shape 4x4x512 and is reshaped to be 64x128

In [ ]:
train_dataset, validation_dataset = ML_model_dataloader(train_data, validation_data, configuration, batch_size=1)
train_batch = train_dataset.take(1)

# extract features vector
recostructed_image = model.predict(train_batch, verbose=0)
extracted_features = encoder_model.predict(train_batch, verbose=0)

for (original_image, label), reshaped, reconstructed in zip(train_batch, extracted_features, recostructed_image):    
    validator.visualize_features_vector(np.squeeze(original_image), reshaped, np.squeeze(reconstructed), checkpoint_path)